In [1]:
with open('./data764/eng.train') as f:
    train_data = f.readlines()
with open('./data764/eng.testb') as f:
    test_data = f.readlines()

In [2]:
train_words = []
train_pos = []
train_chunk = []
train_entity = []
for line in train_data:
    clean_line = line.strip()
    split_line = clean_line.split()
    if (len(split_line) == 4):
        train_words.append(split_line[0])
        train_pos.append(split_line[1])
        train_chunk.append(split_line[2])
        entity = split_line[3]
        if (entity == 'I-LOC' or entity == 'B-LOC'):
            entity = 'LOC'
        elif (entity == 'I-PER' or entity == 'B-PER'):
            entity = 'PER'
        elif (entity == 'I-ORG' or entity == 'B-ORG'):
            entity = 'ORG'
        elif (entity == 'I-MISC' or entity == 'B-MISC'):
            entity = 'MISC'
        train_entity.append(entity)

In [3]:
test_words = []
test_pos = []
test_chunk = []
test_entity = []
for line in test_data:
    clean_line = line.strip()
    split_line = clean_line.split()
    if (len(split_line) == 4):
        test_words.append(split_line[0])
        test_pos.append(split_line[1])
        test_chunk.append(split_line[2])
        entity = split_line[3]
        if (entity == 'I-LOC' or entity == 'B-LOC'):
            entity = 'LOC'
        elif (entity == 'I-PER' or entity == 'B-PER'):
            entity = 'PER'
        elif (entity == 'I-ORG' or entity == 'B-ORG'):
            entity = 'ORG'
        elif (entity == 'I-MISC' or entity == 'B-MISC'):
            entity = 'MISC'
        test_entity.append(entity)

In [4]:
all_chars = []
for word in train_words:
    all_chars.append(word)
for word in test_words:
    all_chars.append(word)

In [5]:
unique_chars = set(all_chars)
#unique_chars

In [6]:
word_dict = {}
for i, char in enumerate(unique_chars):
    #print(f'i:{i}, char:{char}')
    word_dict[char] = i + 1 # + 1 if you want to mask

In [7]:
train_words_numeric = []
test_words_numeric = []
for word in train_words:
    train_words_numeric.append(word_dict[word])
for word in test_words:
    test_words_numeric.append(word_dict[word])

In [8]:
#indexed_words

In [9]:
all_chars = []
for word in train_pos:
    all_chars.append(word)
for word in test_pos:
    all_chars.append(word)
unique_chars = set(all_chars)
#unique_chars
pos_dict = {}
for i, char in enumerate(unique_chars):
    #print(f'i:{i}, char:{char}')
    pos_dict[char] = i + 1 # + 1 if you want to mask
train_pos_numeric = []
test_pos_numeric = []
for word in train_pos:
    train_pos_numeric.append(pos_dict[word])
for word in test_pos:
    test_pos_numeric.append(pos_dict[word])

In [10]:
all_chars = []
for word in train_chunk:
    all_chars.append(word)
for word in test_chunk:
    all_chars.append(word)
unique_chars = set(all_chars)
#unique_chars
chunk_dict = {}
for i, char in enumerate(unique_chars):
    #print(f'i:{i}, char:{char}')
    chunk_dict[char] = i + 1 # + 1 if you want to mask
train_chunk_numeric = []
test_chunk_numeric = []
for word in train_chunk:
    train_chunk_numeric.append(chunk_dict[word])
for word in test_chunk:
    test_chunk_numeric.append(chunk_dict[word])

In [11]:
all_chars = []
for word in train_entity:
    all_chars.append(word)
for word in test_entity:
    all_chars.append(word)
unique_chars = set(all_chars)
#unique_chars
entity_dict = {}
for i, char in enumerate(unique_chars):
#    print(f'i:{i}, char:{char}')
    entity_dict[char] = i + 1 # + 1 if you want to mask
train_entity_numeric = []
test_entity_numeric = []
for word in train_entity:
    train_entity_numeric.append(entity_dict[word])
for word in test_entity:
    test_entity_numeric.append(entity_dict[word])
    

In [12]:
import numpy as np
train_feature = np.stack((train_words_numeric, train_pos_numeric, train_chunk_numeric), axis=1)
test_feature = np.stack((test_words_numeric, test_pos_numeric, test_chunk_numeric), axis=1)
train_feature

array([[21039,    36,     5],
       [24716,    14,    11],
       [ 9530,    10,     6],
       ...,
       [11624,    23,    11],
       [  927,    14,    11],
       [  269,    23,    11]])

In [13]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(train_feature, train_entity_numeric).predict(train_feature)

right = [0] * len(entity_dict)
wrong = [0] * len(entity_dict)

for x in range(0, len(train_entity_numeric)):
    if y_pred[x] == train_entity_numeric[x]:
        right[train_entity_numeric[x] - 1] = right[train_entity_numeric[x] - 1] + 1
    else:
        wrong[train_entity_numeric[x] - 1] = wrong[train_entity_numeric[x] - 1] + 1

for x in range(0, len(right)):
    print("Accuracy (", right[x], "/", right[x]+wrong[x], "): ", right[x]/(right[x]+wrong[x]))
    
print(entity_dict)

print("Number of mislabeled points out of a total %d points : %d. %f"
 % (train_feature.shape[0],(train_entity_numeric != y_pred).sum(), ((train_entity_numeric != y_pred).sum() / train_feature.shape[0])))

Accuracy ( 0 / 10025 ):  0.0
Accuracy ( 165070 / 170524 ):  0.9680162323192043
Accuracy ( 0 / 4593 ):  0.0
Accuracy ( 0 / 8297 ):  0.0
Accuracy ( 10385 / 11128 ):  0.9332314881380301
{'ORG': 1, 'O': 2, 'MISC': 3, 'LOC': 4, 'PER': 5}
Number of mislabeled points out of a total 204567 points : 29112. 0.142310


In [14]:
y_pred = gnb.fit(train_feature, train_entity_numeric).predict(test_feature)

right = [0] * len(entity_dict)
wrong = [0] * len(entity_dict)

for x in range(0, len(test_entity_numeric)):
    if y_pred[x] == test_entity_numeric[x]:
        right[test_entity_numeric[x] - 1] = right[test_entity_numeric[x] - 1] + 1
    else:
        wrong[test_entity_numeric[x] - 1] = wrong[test_entity_numeric[x] - 1] + 1

for x in range(0, len(right)):
    if (right[x] + wrong[x]) > 0:
        print("Accuracy (", right[x], "/", right[x]+wrong[x], "): ", right[x]/(right[x]+wrong[x]))
    
print(entity_dict)

print("Number of mislabeled points out of a total %d points : %d. %f"
 % (test_feature.shape[0],(test_entity_numeric != y_pred).sum(), ((test_entity_numeric != y_pred).sum() / test_feature.shape[0])))

Accuracy ( 0 / 2496 ):  0.0
Accuracy ( 36940 / 38554 ):  0.9581366395185973
Accuracy ( 0 / 918 ):  0.0
Accuracy ( 0 / 1925 ):  0.0
Accuracy ( 2598 / 2773 ):  0.9368914532996755
{'ORG': 1, 'O': 2, 'MISC': 3, 'LOC': 4, 'PER': 5}
Number of mislabeled points out of a total 46666 points : 7128. 0.152745


In [15]:
len(y_pred)

46666

In [16]:
len(test_entity_numeric)

46666

In [17]:
set(y_pred)

{2, 5}